In [ ]:
#first convert dicom files to nifti using dcm2niix
#https://github.com/rordenlab/dcm2niix

In [9]:
from __future__ import division, print_function
from collections import defaultdict
import os, pickle, sys
import shutil
from functools import partial
import numpy as np
from distutils.util import strtobool
from random import shuffle
import matplotlib
import nibabel as nb
from pathlib import Path
import cv2
import pickle
import glob
import h5py
import zipfile
import io
import os.path
import matplotlib.pyplot as plt
import SimpleITK as sitk
import ants


home_dir = '' #directory of nifti files 
new_dir = '' #directory to save files

def retreive_filelist(extension, path, include_subfolders=True):
    """
    Returns a list with all the files of a certain type in path
    :param extension: what extension to search for
    :param include_subfolders: whether to include subfolders
    :param path: specified path. otherwise use data root
    :return:
    """
    if include_subfolders:
        extension = ('**/*.%s' % extension)

    # Otherwise just search this folder
    else:
        extension = ('*.%s' % extension)

    # Join the pathnames
    path = os.path.join(path, extension)
    print(path)
    print(glob.glob(path))

    # Return the list of filenames
    return glob.glob(path)



In [10]:
def call_files ():
    filename = retreive_filelist('nii.gz', home_dir, include_subfolders=True)
    print(len(filename), ' files found. Examples: ', filename)
    for file in filename:
        study = file.split('/')[8] #choose file suffix denoting study ID
        mask_file = file.split('.')[0] + '_Label.nii.gz' #choose file suffix denoting segmentation
        print(mask_file)
        try:
            scan = ants.image_read(file) #read scan image
            mask = ants.image_read(mask_file) #read segmentation
            print(mask)
            print('loading', file)
        except:
            print('unable to load', file)
            continue
        mask_array = mask.numpy() #convert to numpy
        if np.min(mask_array[np.nonzero(mask_array)]) > 1: #selects which segmentation to use if multi-segmented image
            mask_1 = mask.new_image_like(mask_array)
            masked_image = ants.mask_image(scan, mask_1, 2, binarize=False) #apply segmentation to the scan. Set binarize to true if you want a binary mask
            scan = ants.threshold_image(scan, low_thresh=0, high_thresh=100, inval=1, outval=0, binary=False) #thresholds CT image to help skull stripping to work better. This is not necessary for MRI.
            ants.image_write(masked_image, new_dir + study + '_Label.nii.gz') #writes masks and scan to new directory 
            ants.image_write(scan, new_dir + study + '.nii.gz')
        else:
            mask_1 = mask.new_image_like(mask_array)
            masked_image = ants.mask_image(scan, mask_1, 1, binarize=False)
            scan = ants.threshold_image(scan, low_thresh=0, high_thresh=100, inval=1, outval=0, binary=False)
            ants.image_write(masked_image, new_dir + study + '_Label.nii.gz')
            ants.image_write(scan, new_dir + study + '.nii.gz')
     
call_files()

   
   

**/*.nii.gz
[]
0  files found. Examples:  []


In [11]:
def skull_strip (): #loads scans and applies skull stripping. Download here: https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/
    filename = retreive_filelist('nii.gz', new_dir, include_subfolders=False)
    print(len(filename), ' files found. Examples: ', filename)
    for file in filename:
        study = file.split('/')[5].split('.')[0]
        if not file.endswith('_Label.nii.gz'):
            print(file)
            mybet = fsl.BET() 
            result = mybet.run(in_file=file, out_file=new_dir + study + '.nii.gz', frac=0.1)

skull_strip()


*.nii.gz
[]
0  files found. Examples:  []


In [12]:
def resize_volume(img): #downsamples image. Recommend 256 x 256 x 256
    
    desired_depth = 256
    desired_width = 256
    desired_height = 256

    current_depth = img.shape[0]
    current_width = img.shape[1]
    current_height = img.shape[2]
 
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    
    img = ndimage.zoom(img, (depth_factor, width_factor, height_factor), order=1)
    return img

def register_files ():
    filename = retreive_filelist('nii.gz', home_dir, include_subfolders=False)
   
    print(len(filename), ' files found. Examples: ', filename)
    for file in filename:
        study = file.split('/')[7].split('.')[0]
        mask = file.split('.')[0] + '_Label.nii.gz'
        try:
            scan = ants.image_read(file)
            mask1 = ants.image_read(mask)
            print('loading', file)
        except:
            print('unable to load', file)
            continue
        template = ants.image_read('') #loads population based template see https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3376197/
        mytx = ants.registration(fixed=template, moving=scan, type_of_transform='SyN' ) #registers using Symmetric Normalization method
        mywarpedimage = ants.apply_transforms( fixed=template, moving=scan,
                                               transformlist=mytx['fwdtransforms'] )
        mywarpedmask = ants.apply_transforms( fixed=template, moving=mask1,
                                               transformlist=mytx['fwdtransforms'] )
        mywarpedmasknp = mywarpedmask.numpy() 
        mywarpedmaskresize = resize_volume(mywarpedmasknp) #downsample image
        mywarpedmaskants = ants.from_numpy(mywarpedmaskresize)
        mywarpedimagenp = mywarpedimage.numpy()
        mywarpedimageresize = resize_volume(mywarpedimagenp)
        mywarpedimageants = ants.from_numpy(mywarpedimageresize)
        print(mywarpedimageresize.shape)
        ants.image_write(mywarpedimage, new_dir + study + '.nii.gz') #saves registered image
        #ants.image_write(mywarpedimage, new_dir + study + '.tiff') #can also save as tiff files if desired
        ants.image_write(mywarpedmask, new_dir + study + '_Label.nii.gz') #saves registered mask
        #ants.image_write(mywarpedmask, new_dir + study + '_Label.tiff')
        
register_files()


*.nii.gz
[]
0  files found. Examples:  []


In [ ]:
import os
import sys
import copy
from tqdm import tqdm
import nibabel as nib
from nibabel.testing import data_path

## Flip brain lesions across midline, code obtained from https://github.com/hokifung/3D-Brain-Midsagittal-Flip.
## Either choose a separate folder to save lesions on left side and flip to right or vice versa.
## All lesions should be on same hemisphere.

supported_extensions = ('nii', 'gz', 'img', 'gii', 'mnc', 'mgh', 'REC')

def MirrorVoxels(original_data):
    for slice_number in range(0, int(original_data.shape[0] / 2)):
            # Copy Data from Memory (Incase Numpy Uses Shallow Copy)
            temporary_slice = copy.deepcopy(original_data[slice_number, :, :])
            # Calculate Mirror Index
            mirror_idx = original_data.shape[0] - 1 - slice_number
            # Offset Additional 1 from Indexing
            original_data[slice_number,:,:] = copy.deepcopy(original_data[mirror_idx,:,: ])
            # Return Temporary Slice Data to Right Side
            original_data[mirror_idx,:,:] = copy.deepcopy(temporary_slice)

input_path  = ''
output_path = ''

files = os.listdir(path=input_path)
for nfile in tqdm(range(len(files))):
    if files[nfile].lower().endswith(supported_extensions) == False: #sanity check
    continue
    output_name = output_path + "/" + files[nfile]
    filepath = input_path + "/" + files[nfile]
    main = nib.load(filepath)
    header = main.header
    main_data = main.get_fdata() 
    edited_data = copy.deepcopy(main_data)
    MirrorVoxels(edited_data) 
    mirror = nib.Nifti1Image(edited_data, main.affine, main.header)
    nib.save(mirror, output_name)
 
    #Announce
    print("Completed : Files Exported to /{}".format(output_path))

In [ ]:
def smooth_masks(): #smooth the masks using SITK curvature flow 
    filename = retreive_filelist('nii.gz', new_dir, include_subfolders=False)
    print(len(filename), ' files found. Examples: ', filename)
    for file in filename:
        study = file.split('/')[6].split('.')[0]
        mask = file.split('.')[0] + '_Label.nii.gz'
        try:
            #scan = ants.image_read(file) #load files
            mask = ants.image_read(mask) #load files
            print('loading', file)
        except:
            print('unable to load', file)
            continue
        mask_array = mask.numpy() #convert to numpy 
        mask_array = sitk.GetImageFromArray(mask_array)
        mask_array = sitk.CurvatureFlow(image1=mask_array,
                                        timeStep=0.125,
                                        numberOfIterations=5)
        thresholdFilter = sitk.ThresholdImageFilter()
        thresholdFilter.SetLower(0)
        thresholdFilter.SetUpper(150)
        thresholdFilter.SetOutsideValue(0)
        mask_array = thresholdFilter.Execute(mask_array)
        mask_array = sitk.GetArrayFromImage(mask_array)
        img = nb.Nifti1Image(mask_array, affine=np.eye(4))
        nb.save(img, file)
smooth_masks()

In [ ]:
home_dir = ''
new_dir = ''

def mean_image (): #creates the euclidean mean template and reference scan to visualize
    scans=[]
    masks=[]
    filename = retreive_filelist('nii.gz', new_dir, include_subfolders=False)
    print(len(filename), ' files found. Examples: ', filename)
    for file in filename:
        study = file.split('/')[6].split('.')[0]
        mask = file.split('.')[0] + '_Label.nii.gz'
        try:
            scan = ants.image_read(file)
            mask = ants.image_read(mask)
            print('loading', file)
        except:
            print('unable to load', file)
            continue
        mask_array = mask.numpy() #convert to numpy 
        scan_array = scan.numpy()
        masks.append(mask_array)
        scans.append(scan_array)
    masks = np.concatenate(masks, axis=0).reshape(-1, 256, 256, 256) #concatenate the masks
    scans = np.concatenate(scans, axis=0).reshape(-1, 256, 256, 256) #concatenate the scans
    masksmean = np.mean(masks, axis=0)
    scansmean = np.mean(scans, axis=0)
    mean_mask = masksmean.T
    mean_scan = scansmean.T
    img = nb.Nifti1Image(masksmean, affine=np.eye(4))
    nb.save(img, '...template256.nii') #save the template
    img2 = nb.Nifti1Image(scansmean, affine=np.eye(4))
    nb.save(img2, '...meanscan256.nii') #save the scan
mask_mean()


In [ ]:
home_dir = ''
new_dir = ''

def crop(): #crops the images to desired size to remove excess  blank space. You also need to crop the Euclidean mean template and the population mean template used for registration with same parameters and save these separately for later visualizations.
    filename = retreive_filelist('nii.gz', new_dir, include_subfolders=False)
    print(len(filename), ' files found. Examples: ', filename)
    for file in filename:
        study = file.split('/')[8].split('.')[0]
        mask = file.split('.')[0] + '_Label.nii.gz'
        reader = sitk.ImageFileReader() #loads the mask
        reader.SetFileName(mask)
        template = reader.Execute()
        array_mask = sitk.GetArrayFromImage(mask) #converts to numpy format
        print(array_mask.shape)
        maskcrop = array_mask[90:200, 30:220, 50:160] #selects parameters for cropping [x, y, z]
        print(maskcrop.shape)
        maskcrop1 = sitk.GetImageFromArray(maskcrop) #converts back to image format
        outputfile = (new_dir + study + '.nii')
        print(outputfile)
        sitk.WriteImage(maskcrop1, outputfile)
crop()
        